In [1]:
import re
import json
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [4]:
with open('SENT-COVID.json') as file:
    data = json.load(file)
    
pd.options.mode.chained_assignment = None                                         
pd.set_option('display.max_colwidth',None)   

df = pd.DataFrame(data)
df[['Tweet','clean_tweet','Label']].head(10)

,Tweet,clean_tweet,Label
0,1401047081121353728-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,Protocolo de COVID !!!!,NEUTRO
1,1258159310162595843-#COVID19 #QuedateEnCasa en Morelia Centro,COVID QuedateEnCasa en Morelia Centro,NEUTRO
2,1272748988626862082-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO
3,1349385638722883585-@sororavirus Creo en todo y nada. 💜,Creo en todo y nada. 💜,NEUTRO
4,1360615587114844161-@GobiernoMX había prometido 389,había prometido,NEGATIVO
5,1384487745033162755-Vacunación en Tonalá,Vacunación en Tonalá,POSITIVO
6,1335343584875327494-No es cierto lo que dice @WHO,No es cierto lo que dice,NEGATIVO
7,1257755579101003776-Participa en familia con esta entretenida actividad #MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.#aquiestamos #IED #QuedateEnCasa https://t.co/BQZTpxwca2,Participa en familia con esta entretenida actividad MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.aquiestamos IED QuedateEnCasa,POSITIVO
8,1317288163065401345-@lalitogonzaalez Acabar con el Coronavirus.,Acabar con el Coronavirus.,POSITIVO
9,1255373406624776194-#InternationalDanceDay #DID #ENCASA https://t.co/crIQOdoPgG,InternationalDanceDay DID ENCASA,NEUTRO


In [5]:
len(df)

4572

In [7]:
sia = SentimentIntensityAnalyzer()

def get_polarity(text):
    return sia.polarity_scores(text)

df["sentiment"] = df["clean_tweet"].apply(get_polarity)
df['sentiment'].head()

0           {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
1           {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2    {'neg': 0.066, 'neu': 0.934, 'pos': 0.0, 'compound': -0.296}
3           {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
4           {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Name: sentiment, dtype: object

In [10]:
def score(x):
  if x > 0:
    return 'POSITIVO'
  elif x == 0:
    return 'NEUTRO'
  else:
    return 'NEGATIVO'

df["polarity"] = df["clean_tweet"].apply(lambda i: sia.polarity_scores(i)['compound'])
df['score'] = df['polarity'].apply(score)
df[['clean_tweet','Label','polarity','score']].head(10)

,clean_tweet,Label,polarity,score
0,Protocolo de COVID !!!!,NEUTRO,0.000,NEUTRO
1,COVID QuedateEnCasa en Morelia Centro,NEUTRO,0.000,NEUTRO
2,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO,-0.296,NEGATIVO
3,Creo en todo y nada. 💜,NEUTRO,0.000,NEUTRO
4,había prometido,NEGATIVO,0.000,NEUTRO
5,Vacunación en Tonalá,POSITIVO,0.000,NEUTRO
6,No es cierto lo que dice,NEGATIVO,-0.296,NEGATIVO
7,Participa en familia con esta entretenida actividad MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.aquiestamos IED QuedateEnCasa,POSITIVO,0.000,NEUTRO
8,Acabar con el Coronavirus.,POSITIVO,0.000,NEUTRO
9,InternationalDanceDay DID ENCASA,NEUTRO,0.000,NEUTRO


In [11]:
df['agreement'] = 0

for i in df.index:
    if df['Label'][i] == df['score'] [i]:
        df['agreement'][i] = 'match'
    elif df['Label'][i]=='POSITIVO' and df['score'][i]=='NEGATIVO':
        df['agreement'][i] = 'opp'
    elif df['score'][i]=='POSITIVO'and df['Label'][i]=='NEGATIVO':
        df['agreement'][i] = 'opp'
    else:
        df['agreement'][i] = 'not match'
        
df['matrix_value'] = 0

for i in df.index:
    if df['score'][i] =='POSITIVO' and df['Label'] [i] =='POSITIVO':
        df['matrix_value'][i] = 'VP'
    elif df['score'][i] =='NEGATIVO' and df['Label'] [i] =='NEGATIVO':
        df['matrix_value'][i] = 'VN'
    elif df['score'][i] =='POSITIVO' and df['Label'] [i] !='POSITIVO':
        df['matrix_value'][i] = 'FP'
    elif df['score'][i] =='NEGATIVO' and df['Label'] [i] !='NEGATIVO': 
        df['matrix_value'][i] = 'FN'
        
df[['clean_tweet','Label','polarity','score','agreement','matrix_value']].head()          

,clean_tweet,Label,polarity,score,agreement,matrix_value
0,Protocolo de COVID !!!!,NEUTRO,0.000,NEUTRO,match,0
1,COVID QuedateEnCasa en Morelia Centro,NEUTRO,0.000,NEUTRO,match,0
2,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO,-0.296,NEGATIVO,opp,FN
3,Creo en todo y nada. 💜,NEUTRO,0.000,NEUTRO,match,0
4,había prometido,NEGATIVO,0.000,NEUTRO,not match,0


In [14]:
def accuracy(vp, vn, fp, fn):
    ex = (vp+vn)/(vp+fp+fn+vn) #TP+TN/TP+FP+FN+TN
    return ex

def precision(vp, fp):
    pr = vp/(vp+fp) #TP/TP+FP
    return pr

def recall(vp, fn):
    se = vp/(vp+fn) #TP/TP+FN
    return se

def f1(vn, fp):
    es = 2*(recall(vp, fn)*precision(vp, fp)) / (recall(vp, fn)+precision(vp, fp))
    return es

vp=df['matrix_value'].str.contains('VP').value_counts()[True]
vn=df['matrix_value'].str.contains('VN').value_counts()[True]
fp=df['matrix_value'].str.contains('FP').value_counts()[True]
fn=df['matrix_value'].str.contains('FN').value_counts()[True]

In [15]:
print('Accuracy: '+ str(accuracy(vp, vn, fp, fn)))
print('Precision: '+ str(precision(vp, fp)))
print('Recall: '+ str(recall(vp, fn)))
print('f1-score: '+ str(f1(vn, fp)))

Accuracy: 0.4773489932885906
Precision: 0.3093220338983051
Recall: 0.13696060037523453
f1-score: 0.18985695708712613


In [16]:
df['agreement'].value_counts()

match        2194
not match    2166
opp           212
Name: agreement, dtype: int64

In [17]:
df['matrix_value'].value_counts()

0     3380
VN     496
FN     460
FP     163
VP      73
Name: matrix_value, dtype: int64

In [18]:
df['Label'].value_counts()

NEUTRO      2036
NEGATIVO    1560
POSITIVO     976
Name: Label, dtype: int64

In [19]:
df['score'].value_counts()

NEUTRO      3380
NEGATIVO     956
POSITIVO     236
Name: score, dtype: int64